In [1]:
## imporrt libarary 
import os
import pandas as pd
import numpy as np

#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform

## sklearn module
import pickle
import xgboost
from xgboost import XGBRegressor


### Authenticate to GCP using Services Account 

- Depending on your Jupyter environment, you may have to manually 

- Call "sa-development.json" as teh auth to GCP 

- Adjust your path where the all code stored in you local 

In [2]:
## set up authentication using services account 
# Authenticate using service account
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../jaya-000/sa-development.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../luqman_022/sa-development.json"

In [3]:
project_id = 'dti-ds'
dataset_id = 'luqman_dataset_022'
table_id = 'test_data'
region = 'us-central1'
bucket_name = 'luqman_gcs_022'
blob_name = 'data/test_data.csv'

# Load and Data Preparation

In [4]:
#Load data from Big Query
client = bigquery.Client(project=project_id)

# query 
query_job = client.query(f"""select * from {dataset_id}.{table_id}""")
df_test = query_job.result().to_dataframe()
df_test.head()

/home/luqmanilman/miniconda3/envs/lqliciouscapstone/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,hour,humidity,weather,holiday,season,atemp,month,year,dayname
0,14,0.41,clear,0,winter,0.2121,1,2011,4
1,14,0.28,clear,0,winter,0.2879,1,2011,2
2,11,0.44,cloudy,0,winter,0.4394,2,2011,3
3,9,0.64,clear,0,winter,0.1818,2,2011,5
4,19,0.87,cloudy,1,winter,0.2121,2,2011,0


> Try predict with raw data

In [5]:
filename = 'model_capstone4.pkl'
loaded_model = pickle.load(open(filename, 'rb'))

In [6]:
filename = 'model_capstone4_transformer.pkl'
loaded_model_transformer = pickle.load(open(filename, 'rb'))

In [7]:
x_preprocess = loaded_model_transformer.transform(df_test)

In [8]:
predict1 = loaded_model.predict(x_preprocess)

In [9]:
df_test['y_pred'] = predict1

In [10]:
df_test

,hour,humidity,weather,holiday,season,atemp,month,year,dayname,y_pred
0,14,0.41,clear,0,winter,0.2121,1,2011,4,56.056931
1,14,0.28,clear,0,winter,0.2879,1,2011,2,56.884987
2,11,0.44,cloudy,0,winter,0.4394,2,2011,3,93.547791
3,9,0.64,clear,0,winter,0.1818,2,2011,5,49.502327
4,19,0.87,cloudy,1,winter,0.2121,2,2011,0,38.156567
...,...,...,...,...,...,...,...,...,...,...
95,6,0.49,cloudy,0,fall,0.3182,11,2012,5,24.220985
96,12,0.46,cloudy,0,fall,0.3939,11,2012,4,273.744843
97,11,0.52,clear,0,winter,0.2273,12,2012,6,139.229919
98,9,0.64,cloudy,0,winter,0.2121,12,2012,0,189.032730


> Try predict with Extreme Values

In [11]:
new_cust = [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.69, 0.2576, 11.0, 2012.0, 0.0]
new_cust

[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.69, 0.2576, 11.0, 2012.0, 0.0]

In [12]:
new_cust2 = [0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 17.0, 0.28, 0.6364, 9.0, 2012.0, 1.0]
new_cust2

[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 17.0, 0.28, 0.6364, 9.0, 2012.0, 1.0]

In [13]:
loaded_model.predict([new_cust,new_cust2])

array([ 14.12539, 963.35284], dtype=float32)

In [14]:
from google.cloud import bigquery

table_id = 'data_test_new'

# Construct a bigquery client object
client = bigquery.Client()

# define the full table id
table_full_id = f"{client.project}.{dataset_id}.{table_id}"

df_test.columns = ['_'.join(i.split(' ')) for i in df_test.columns]

job = client.load_table_from_dataframe(df_test,table_full_id)

job.result()
print(f"Loaded {job.output_rows} rows into {table_full_id}")

Loaded 100 rows into dti-ds.luqman_dataset_022.data_test_new
